**Rotator Performance Turning**

This Jupyter notebook is used to turn the controller parameters by Tekniker.
The test will be performed on summit with the real hardware.
Before that, the Tucson test stand is used to develop the test procedure.
The rotator commandable SAL component (CSC) is used to control the rotator hardware.
The published event and telemetry are stored in the engineering facility database (EFD).

It is noted that the **MTMount** telemetry data is required.
Otherwise, the rotator CSC will put the low-level controller into the **Fault** state.

The **setup** notebook can be used to bring up rotator and mount/CCW.

**Import Modules**

In [1]:
%matplotlib inline
%matplotlib widget
import asyncio
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lsst.ts import salobj
from lsst.ts import simactuators
from lsst_efd_client import EfdClient

**Custom Functions**

In [2]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)

In [3]:
def script_log_start(controller_script, test_title, test_detail):
    """Log the start of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"START -- {test_title} -- LVV-T2539 -- {test_detail} -- Starting Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [25]:
def script_log_stop(controller_script, test_title, test_detail):
    """Log the stop of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"STOP -- {test_title} -- LVV-T2539 -- {test_detail} -- Stopping Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [57]:
async def track_targets(csc):
    """Track the targets.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    
    for sign in (1, -1):
        await track_target(csc, 0, sign * 0.5, sign * 0.068)
        await move_origin(csc)

        await track_target(csc, 0, sign * 0.8, sign * 0.1)
        await move_origin(csc)

        await track_target(csc, 0, sign * 0.5, sign * 0.03)
        await move_origin(csc)

        await track_target(csc, sign * 10, sign * 11, sign * 0.068)
        await move_origin(csc)

        await track_target(csc, sign * 15, sign * 15.5, sign * 0.03)
        await move_origin(csc)


In [14]:
async def track_target(csc, start_position, end_position, velocity):
    """Track the target.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    start_position : float
        Starting position of ramp (deg).
    end_position : float
        Ending position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    """    

    await make_sure_csc_enabled(csc)
    
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator = simactuators.RampGenerator(
        [start_position], [end_position], [velocity], track_advance_time)

    await csc.cmd_trackStart.start(timeout=15)
    await asyncio.sleep(0.1)
    for positions, velocities, tai in ramp_generator():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)

In [13]:
async def move_origin(csc):
    """Move to the origin.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    sleep_time = 3
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    # Workaround the problem of MOOG state machine to decide a new point-to-point movement
    await csc.cmd_move.set_start(position=0.1, timeout=60)
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    await csc.cmd_move.set_start(position=0, timeout=60)
    await asyncio.sleep(sleep_time)

In [12]:
async def make_sure_csc_enabled(csc):
    """Make sure the CSC is enabled.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    data = await csc.evt_summaryState.aget(timeout=30)
    if data.summaryState != salobj.State.ENABLED:
        await salobj.set_summary_state(csc, salobj.State.ENABLED, timeout=10)

**Instantiate the EFD and CSC**

Need to make sure **ospl** is running for the data distribution system (DDS) communication.

In [15]:
is_summit = False
efd_client = retrieve_efd_client(is_summit=is_summit)
domain = salobj.Domain()
rotator = salobj.Remote(domain, "MTRotator")
await rotator.start_task
await rotator.cmd_setLogLevel.set_start(level=10)

**MTMount** controller is used to run the camera cable wrapper (CCW).

In [16]:
mtmount = salobj.Remote(domain, "MTMount")
await mtmount.start_task

state_mtmount = await mtmount.evt_summaryState.aget(timeout=10)
if state_mtmount != salobj.State.ENABLED:
    await salobj.set_summary_state(mtmount, salobj.State.ENABLED, timeout=10)

await mtmount.cmd_enableCameraCableWrapFollowing.set_start()

Set the rotator CSC to be **Enabled** state.

In [17]:
await salobj.set_summary_state(rotator, salobj.State.ENABLED, timeout=10)

[<State.ENABLED: 2>]

**Setup the Script**

In [21]:
time_start_evaluation = datetime.now()
test_title = "Evaluate the Rotator Controller Performance"
script = salobj.Controller("Script", index=10)

# Wait 10 second may help with DDS problems; closing all other kernels may help too
await asyncio.sleep(10)
print(f"Time to start the evaluation is {time_start_evaluation}.")

Time to start the evaluation is 2022-06-01 14:22:32.160050.


**Baseline**

In [23]:
time_start = script_log_start(script, test_title, "Baseline")

In [58]:
await track_targets(rotator)

In [26]:
time_stop = script_log_stop(script, test_title, "Baseline")

**Modify VEL_LOOP_KP**

VEL_LOOP_KP = 106

In [27]:
time_start = script_log_start(script, test_title, "VEL_LOOP_KP = 106")

In [54]:
await track_targets(rotator)

In [28]:
time_stop = script_log_stop(script, test_title, "VEL_LOOP_KP = 106")

VEL_LOOP_KP = 80

In [29]:
time_start = script_log_start(script, test_title, "VEL_LOOP_KP = 80")

In [ ]:
await track_targets(rotator)

In [30]:
time_stop = script_log_stop(script, test_title, "VEL_LOOP_KP = 80")

**Modify VEL_LOOP_TI**

VEL_LOOP_TI = 0.11

In [31]:
time_start = script_log_start(script, test_title, "VEL_LOOP_TI = 0.11")

In [ ]:
await track_targets(rotator)

In [32]:
time_stop = script_log_stop(script, test_title, "VEL_LOOP_TI = 0.11")

VEL_LOOP_TI = 0.09

In [33]:
time_start = script_log_start(script, test_title, "VEL_LOOP_TI = 0.09")

In [ ]:
await track_targets(rotator)

In [34]:
time_stop = script_log_stop(script, test_title, "VEL_LOOP_TI = 0.09")

**Modify POS_LOOP_KV**

POS_LOOP_KV = 34

In [35]:
time_start = script_log_start(script, test_title, "POS_LOOP_KV = 34")

In [ ]:
await track_targets(rotator)

In [36]:
time_stop = script_log_stop(script, test_title, "POS_LOOP_KV = 34")

POS_LOOP_KV = 28

In [37]:
time_start = script_log_start(script, test_title, "POS_LOOP_KV = 28")

In [ ]:
await track_targets(rotator)

In [38]:
time_stop = script_log_stop(script, test_title, "POS_LOOP_KV = 28")

**Modify ENABLE_FFV**

ENABLE_FFV = 0

In [39]:
time_start = script_log_start(script, test_title, "ENABLE_FFV = 0")

In [ ]:
await track_targets(rotator)

In [40]:
time_stop = script_log_stop(script, test_title, "ENABLE_FFV = 0")

**Modify PRELOAD_KP**

PRELOAD_KP = 0.11

In [41]:
time_start = script_log_start(script, test_title, "PRELOAD_KP = 0.11")

In [ ]:
await track_targets(rotator)

In [42]:
time_stop = script_log_stop(script, test_title, "PRELOAD_KP = 0.11")

PRELOAD_KP = 0.09

In [43]:
time_start = script_log_start(script, test_title, "PRELOAD_KP = 0.09")

In [ ]:
await track_targets(rotator)

In [44]:
time_stop = script_log_stop(script, test_title, "PRELOAD_KP = 0.09")

**Modify PRELOAD_TI**

PRELOAD_TI = 5.5

In [45]:
time_start = script_log_start(script, test_title, "PRELOAD_TI = 5.5")

In [ ]:
await track_targets(rotator)

In [46]:
time_stop = script_log_stop(script, test_title, "PRELOAD_TI = 5.5")

PRELOAD_TI = 4.5

In [47]:
time_start = script_log_start(script, test_title, "PRELOAD_TI = 4.5")

In [ ]:
await track_targets(rotator)

In [48]:
time_stop = script_log_stop(script, test_title, "PRELOAD_TI = 4.5")

**Modify PRELOAD_TQ**

PRELOAD_TQ = 0.17

In [49]:
time_start = script_log_start(script, test_title, "PRELOAD_TQ = 0.17")

In [ ]:
await track_targets(rotator)

In [50]:
time_stop = script_log_stop(script, test_title, "PRELOAD_TQ = 0.17")

PRELOAD_TQ = 0.19

In [51]:
time_start = script_log_start(script, test_title, "PRELOAD_TQ = 0.19")

In [ ]:
await track_targets(rotator)

In [52]:
time_stop = script_log_stop(script, test_title, "PRELOAD_TQ = 0.19")